,korean,actual translation,Google translation,DeepL translation
0,아직 3분 시작된 건 아니죠? 그렇죠?,"My three minutes hasn't started yet, has it?","The 3 minutes haven't started yet, right? Right?",NaN
1,"크리스 앤더슨:네, 맘대로 시작하실 수 없습니다.","No, you can't start the three minutes.","Chris Anderson: Yes, you can't just start.",NaN
2,"3분 다시 설정해주세요, 이건 반칙입니다.","Reset the three minutes, that's just not fair.","Please reset the time to 3 minutes, this is a ...",NaN
3,"앨리슨 헌트 : 어머나, 여기 참 냉정하네요","Oh my God, it's harsh up here.","Allison Hunt: Oh my, it's so cold in here.",NaN
4,정말이지 긴장되네요,I mean I'm nervous enough as it is.,I'm really nervous.,NaN


In [ ]:
conda install -c conda-forge pyyaml


In [11]:
conda install spacy

Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.7.0
    latest version: 25.11.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [13]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 74.4 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [14]:
!pip install torch
!pip install transformers
!pip install unbabel-comet
!pip install evaluate



  Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (114 kB)
Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.5
    Uninstalling numpy-2.3.5:
      Successfully uninstalled numpy-2.3.5


In [ ]:
import pandas as pd
import numpy as np
import torch
import transformers
import spacy
from comet import download_model, load_from_checkpoint
import evaluate


All imports worked.


In [23]:
# LOAD DATA
# ------------------------------------------------
results = pd.read_csv("data/korean_translation.csv")
results.head()

,korean,actual translation,Google translation,DeepL translation
0,아직 3분 시작된 건 아니죠? 그렇죠?,"My three minutes hasn't started yet, has it?","The 3 minutes haven't started yet, right? Right?",NaN
1,"크리스 앤더슨:네, 맘대로 시작하실 수 없습니다.","No, you can't start the three minutes.","Chris Anderson: Yes, you can't just start.",NaN
2,"3분 다시 설정해주세요, 이건 반칙입니다.","Reset the three minutes, that's just not fair.","Please reset the time to 3 minutes, this is a ...",NaN
3,"앨리슨 헌트 : 어머나, 여기 참 냉정하네요","Oh my God, it's harsh up here.","Allison Hunt: Oh my, it's so cold in here.",NaN
4,정말이지 긴장되네요,I mean I'm nervous enough as it is.,I'm really nervous.,NaN


In [34]:
INPUT_CSV = "Data/translations.csv"          # your input file
OUTPUT_CSV = "data/translations_scored.csv"  # file to save with scores

SOURCE_COL = 'korean'
REF_COL = 'actual translation'
MT_COL = 'Google translation'

In [35]:
BLEURT_MODEL_NAME = "Elron/bleurt-base-512" 

In [31]:
results.columns

Index(['korean', 'actual translation', 'Google translation',
       'DeepL translation'],
      dtype='object')

In [36]:
for col in [SOURCE_COL, REF_COL, MT_COL]:
    if col not in results.columns:
        raise ValueError(f"Column '{col}' not found in CSV. Found columns: {results.columns.tolist()}")

sources = results[SOURCE_COL].astype(str).tolist()
references = results[REF_COL].astype(str).tolist()
translations = results[MT_COL].astype(str).tolist()

print(f"Loaded {len(results)} rows from {INPUT_CSV}")

Loaded 1981 rows from Data/translations.csv


In [39]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [40]:
# SETUP BLEURT
# ------------------------------------------------

print("Loading BLEURT model...")
bleurt_tokenizer = AutoTokenizer.from_pretrained(BLEURT_MODEL_NAME)
bleurt_model = AutoModelForSequenceClassification.from_pretrained(BLEURT_MODEL_NAME)
bleurt_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bleurt_model.to(device)
print(f"BLEURT running on: {device}")

def bleurt_score_batch(references, candidates, batch_size=16):
    """Compute BLEURT scores for (ref, cand) pairs in batches."""
    assert len(references) == len(candidates)
    scores = []

    for i in range(0, len(references), batch_size):
        refs_batch = references[i:i+batch_size]
        cands_batch = candidates[i:i+batch_size]

        with torch.no_grad():
            inputs = bleurt_tokenizer(
                refs_batch,
                cands_batch,
                padding=True,
                truncation=True,
                return_tensors="pt"
            ).to(device)

            outputs = bleurt_model(**inputs).logits.squeeze(-1)
            scores.extend(outputs.cpu().tolist())

    return scores

print("Scoring with BLEURT...")
bleurt_scores = bleurt_score_batch(references, translations, batch_size=16)
results["bleurt_score"] = bleurt_scores

Loading BLEURT model...
BLEURT running on: cpu
Scoring with BLEURT...


In [41]:
bleurt_scores

[0.10917980968952179,
 -0.6642258167266846,
 -0.5793778896331787,
 -0.9233299493789673,
 0.4433097541332245,
 -0.21976065635681152,
 0.7969740629196167,
 0.43953952193260193,
 -0.18171969056129456,
 0.00017567723989486694,
 0.05871455371379852,
 1.0291458368301392,
 1.0883123874664307,
 0.9429195523262024,
 0.6161854267120361,
 -0.27220067381858826,
 0.38482677936553955,
 -0.26064151525497437,
 0.5982016324996948,
 0.06262756884098053,
 -0.5847036242485046,
 0.24892446398735046,
 0.21101349592208862,
 -0.38176724314689636,
 0.2427753508090973,
 1.0259922742843628,
 0.3380931615829468,
 -0.15787507593631744,
 -0.48409122228622437,
 -0.08715155720710754,
 0.14020170271396637,
 0.04240201413631439,
 0.1873132586479187,
 0.018644899129867554,
 0.33081233501434326,
 -0.38792499899864197,
 1.0180978775024414,
 0.33347055315971375,
 0.49556148052215576,
 -0.01392238587141037,
 0.7330276370048523,
 -0.02371944487094879,
 -0.4358256459236145,
 0.2125760018825531,
 0.1516421139240265,
 0.1601474

In [44]:
print("Loading COMET model (this may download the model the first time)...")
comet_model_path = download_model("Unbabel/wmt22-comet-da")
comet_model = load_from_checkpoint(comet_model_path)

comet_data = [
    {"src": src, "mt": mt, "ref": ref}
    for src, mt, ref in zip(sources, translations, references)
]

print("Scoring with COMET...")
# gpus=0 forces CPU; set gpus=1 if you have a GPU and it's configured
comet_output = comet_model.predict(comet_data, batch_size=16, gpus=1)

comet_seg_scores = comet_output.scores       # per-segment scores
comet_sys_score = comet_output.system_score  # overall system score

results["comet_score"] = comet_seg_scores

print(f"System-level COMET score: {comet_sys_score:.4f}")

# SAVE RESULTS
# ------------------------------------------------
results.to_csv(OUTPUT_CSV, index=False)
print(f"Saved scores to {OUTPUT_CSV}")



Loading COMET model (this may download the model the first time)...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.6.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Encoder model frozen.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Scoring with COMET...


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
Predicting: 0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable 	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
TOKENIZERS_PARALLELISM=(true | false)
Predicting DataLoader 0: 100%|██████████| 124/124 [05:49<00:00,  2.82s/it]


System-level COMET score: 0.8323
Saved scores to data/translations_scored.csv


In [ ]:
# Show a preview
print(results[[SOURCE_COL, REF_COL, MT_COL, "bleurt_score", "comet_score"]].head())